In [415]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import kaleido

df = pd.read_csv("house_data_price_outlier_remove.csv",index_col="Unnamed: 0" )
zips_full = pd.read_csv("Belgium_zip_municipalities.txt", sep="\t")
print(zips_full)
zips = zips_full.iloc[:,1:3]
zips.columns = ["Zip_code", "city"]
pd.options.display.float_format = '{:.0f}'.format
print(df.shape)
df = df.drop_duplicates(["Area", "Price", "State_of_building", "Zip_code","Land_surface", "Garden_surface"])
print(df.shape)
df.to_csv("house_data_ultimate_cleaned")
print(df.columns)
print(zips_full.columns)

     country_code  Zip_code         place_name         admin_name1  \
0              BE      1000          Bruxelles  Bruxelles-Capitale   
1              BE      1020             Laeken  Bruxelles-Capitale   
2              BE      1030         Schaerbeek  Bruxelles-Capitale   
3              BE      1040          Etterbeek  Bruxelles-Capitale   
4              BE      1050            Ixelles  Bruxelles-Capitale   
...           ...       ...                ...                 ...   
2775           BE      5680           Niverlée            Wallonie   
2776           BE      5680            Romerée            Wallonie   
2777           BE      5680  Matagne-La-Petite            Wallonie   
2778           BE      5680  Matagne-La-Grande            Wallonie   
2779           BE      5680          Vaucelles            Wallonie   

     admin_code1  admin_name2 admin_code2 admin_name3  \
0            BRU  Bruxelles (19 communes)         BRU   
1            BRU  Bruxelles (19 communes)    

In [416]:
# Dropping outlier
df.drop(df[df["Zip_code"] == 1404].index,inplace=True)

In [417]:
# Grouping into municipalities.
def rounding_down(x):
    rest = x % 10
    if rest != 0:
        return x - rest
    else:
        return x

df["municipalities"] = df["Zip_code"].apply(lambda x: rounding_down(x))
df["municipalities"]

0       3290
1       2070
2       8800
3       8000
4       8800
        ... 
8328    4280
8329    4040
8330    7390
8331    4470
8332    9290
Name: municipalities, Length: 7801, dtype: int64

In [418]:
# Dropping if only 5 valid price rows
threshold = 5
value_counts = df.municipalities.value_counts()
to_remove = list(value_counts[value_counts <= threshold].index)
df_threshold = df.copy()

for label, row in df_threshold.iterrows():
    if row.municipalities in to_remove:
        df_threshold.drop(index=label, inplace=True)

print(df.shape)

(7801, 19)


In [419]:
# Function top 5
def make_top_5(df, postal_column:str, on_what:str, method:str, top_or_bottom:str) -> str:
    number_of_values = 5
    if top_or_bottom == "top":
        dct = dict()
        postal_on_what = df.groupby(postal_column).agg({on_what:method})
        postal_on_what = postal_on_what.sort_values(on_what, ascending=False)[:number_of_values]
        postal_on_what.reset_index(level=0, inplace=True)
        df_with_cities = postal_on_what.merge(zips, right_on="Zip_code", left_on="municipalities", how="left")
        """print(df_with_cities)"""
        df_with_cities = df_with_cities.drop_duplicates("Zip_code", keep="first")
        df_with_cities = df_with_cities.drop("municipalities", axis=1)
        if on_what == "Price_per_square_meter":
            df_with_cities = df_with_cities[["Zip_code", "city", on_what]]
            df_with_cities.columns = ["Zip_code", "municipality", on_what]
        else:
            df_with_cities = df_with_cities[["Zip_code", "city", on_what]]
            df_with_cities.columns = ["Zip_code", "municipality", f"{on_what}_{method}"]
        df_with_cities.index = [x for x in range(1,number_of_values+1)]
        return df_with_cities
    elif top_or_bottom == "bottom":
        dct = dict()
        postal_on_what = df.groupby(postal_column).agg({on_what:method})
        postal_on_what = postal_on_what.sort_values(on_what, ascending=True)[:number_of_values]
        postal_on_what.reset_index(level=0, inplace=True)
        df_with_cities = postal_on_what.merge(zips, right_on="Zip_code", left_on="municipalities", how="left")
        """print(df_with_cities)"""
        df_with_cities = df_with_cities.drop_duplicates("Zip_code", keep="first")
        df_with_cities = df_with_cities.drop("municipalities", axis=1)
        if on_what == "Price_per_square_meter":
            df_with_cities = df_with_cities[["Zip_code", "city", on_what]]
            df_with_cities.columns = ["Zip_code", "municipality", on_what]
        else:
            df_with_cities = df_with_cities[["Zip_code", "city", on_what]]
            df_with_cities.columns = ["Zip_code", "municipality", f"{on_what}_{method}"]
        df_with_cities.index = [x for x in range(1,number_of_values+1)]
        return df_with_cities
    else:
        raise ValueError("""
Unexpected argument.
Expected 'top' or 'bottom'""")

In [420]:
# Most expensive mean - By price - Belgium

top5_price_Belgium_mean = make_top_5(df_threshold, "municipalities", "Price", "mean", "top")
top5_price_Belgium_mean

,Zip_code,municipality,Price_mean
1,1640,Rhode-Saint-Genèse,1682692
2,1180,Uccle,1537943
3,8300,Knokke-Heist,1373209
4,1950,Kraainem,1276667
5,1050,Ixelles,1238990


In [421]:
# Most expensive median - By price - Belgium
top5_price_Belgium_median = make_top_5(df_threshold, "municipalities", "Price", "median", "top")
top5_price_Belgium_median

,Zip_code,municipality,Price_median
1,1640,Rhode-Saint-Genèse,1745000
2,1180,Uccle,1492500
3,8300,Knokke-Heist,1190000
4,1050,Ixelles,1070000
5,2000,Antwerpen,995000


In [422]:
# Most expensive mean - By € / SQM - Belgium
top5_sqm_Belgium_mean = make_top_5(df_threshold, "municipalities", "Price_per_square_meter", "mean", "top")
top5_sqm_Belgium_mean

,Zip_code,municipality,Price_per_square_meter
1,8300,Knokke-Heist,5458
2,1180,Uccle,4131
3,1640,Rhode-Saint-Genèse,3883
4,1150,Woluwe-Saint-Pierre,3848
5,1970,Wezembeek-Oppem,3785


In [423]:
# Only Flanders DataFrame
df_threshold_Flanders = df_threshold[df_threshold["Regions"] == "Flanders"]


In [424]:
# Most expensive mean - By price - Flanders
top5_price_Flanders_mean = make_top_5(df_threshold_Flanders, "municipalities", "Price", "mean", "top")
top5_price_Flanders_mean

,Zip_code,municipality,Price_mean
1,1640,Rhode-Saint-Genèse,1682692
2,8300,Knokke-Heist,1373209
3,1950,Kraainem,1276667
4,3080,Tervuren,1149362
5,2970,'S Gravenwezel,1111000


In [425]:
# Most expensive median - By price - Flanders
top5_price_Flanders_median = make_top_5(df_threshold_Flanders, "municipalities", "Price", "median", "top")
top5_price_Flanders_median

,Zip_code,municipality,Price_median
1,1640,Rhode-Saint-Genèse,1745000
2,8300,Knokke-Heist,1190000
3,2000,Antwerpen,995000
4,3140,Keerbergen,935000
5,1560,Hoeilaart,920000


In [426]:
# Most expensive mean - By € / SQM - Flanders
top5_sqm_Flanders_mean = make_top_5(df_threshold_Flanders, "municipalities", "Price_per_square_meter", "mean", "top")
top5_sqm_Flanders_mean

,Zip_code,municipality,Price_per_square_meter
1,8300,Knokke-Heist,5458
2,1640,Rhode-Saint-Genèse,3883
3,1970,Wezembeek-Oppem,3785
4,3000,Leuven,3557
5,1950,Kraainem,3439


In [427]:
# Only Wallonia DataFrame
df_threshold_Wallonia = df_threshold[df_threshold["Regions"] == "Wallonia"]


In [428]:
# Most expensive mean - By price - Wallonia
top5_price_Wallonia_mean = make_top_5(df_threshold_Wallonia, "municipalities", "Price", "mean", "top")
top5_price_Wallonia_mean

,Zip_code,municipality,Price_mean
1,1380,Lasne,1040419
2,1410,Waterloo,994451
3,7520,Templeuve,792500
4,1310,La Hulpe,759292
5,6280,Gerpinnes,743875


In [429]:
# Most expensive median - By price - Wallonia
top5_price_Wallonia_median = make_top_5(df_threshold_Wallonia, "municipalities", "Price", "median", "top")
top5_price_Wallonia_median

,Zip_code,municipality,Price_median
1,1380,Lasne,895000
2,7520,Templeuve,842500
3,1410,Waterloo,820000
4,1370,Jodoigne,589500
5,1330,Rixensart,580000


In [430]:
# Most expensive mean - By € / SQM - Wallonia
top5_sqm_Wallonia_mean = make_top_5(df_threshold_Wallonia, "municipalities", "Price_per_square_meter", "mean", "top")
top5_sqm_Wallonia_mean


,Zip_code,municipality,Price_per_square_meter
1,1380,Lasne,3170
2,1410,Waterloo,3053
3,1330,Rixensart,2951
4,1440,Braine-Le-Château,2665
5,1340,Ottignies-Louvain-La-Neuve,2650


In [431]:
# Least expensive mean - By price - Belgium
low5_price_Belgium_mean = make_top_5(df_threshold, "municipalities", "Price", "mean", "bottom")
low5_price_Belgium_mean

,Zip_code,municipality,Price_mean
1,6020,Dampremy,129800
2,7340,Colfontaine,130429
3,7390,Quaregnon,138768
4,6200,Châtelineau,149582
5,6870,Saint-Hubert,152250


In [432]:
# Least expensive median - By price - Belgium
low5_price_Belgium_median = make_top_5(df_threshold, "municipalities", "Price", "median", "bottom")
low5_price_Belgium_median

,Zip_code,municipality,Price_median
1,6020,Dampremy,109950
2,7340,Colfontaine,110000
3,6870,Saint-Hubert,122500
4,7390,Quaregnon,124500
5,7370,Dour,141250


In [433]:
# Least expensive mean - By € / SQM - Belgium
low5_sqm_Belgium_mean = make_top_5(df_threshold, "municipalities", "Price_per_square_meter", "mean", "bottom")
low5_sqm_Belgium_mean

,Zip_code,municipality,Price_per_square_meter
1,6020,Dampremy,733
2,6870,Saint-Hubert,802
3,6590,Momignies,868
4,7370,Dour,872
5,6690,Vielsalm,877


In [434]:
# Least expensive mean - By price - Flanders
low5_price_Flanders_mean = make_top_5(df_threshold_Flanders, "municipalities", "Price", "mean", "bottom")
low5_price_Flanders_mean

,Zip_code,municipality,Price_mean
1,8950,Heuvelland,224167
2,3640,Kinrooi,238889
3,8520,Kuurne,246125
4,8650,Houthulst,249896
5,8920,Langemark,259488


In [435]:
# Least expensive median - By price - Flanders
low5_price_Flanders_median = make_top_5(df_threshold_Flanders, "municipalities", "Price", "median", "bottom")
low5_price_Flanders_median

,Zip_code,municipality,Price_median
1,8950,Heuvelland,224500
2,8650,Houthulst,232000
3,2660,Hoboken,239000
4,8580,Avelgem,239500
5,3640,Kinrooi,241000


In [436]:
# Least expensive mean - By € / SQM - Flanders
low5_sqm_Flanders_mean = make_top_5(df_threshold_Flanders, "municipalities", "Price_per_square_meter", "mean", "bottom")
low5_sqm_Flanders_mean

,Zip_code,municipality,Price_per_square_meter
1,8880,Ledegem,1304
2,8920,Langemark,1319
3,3640,Kinrooi,1366
4,8650,Houthulst,1422
5,8580,Avelgem,1454


In [437]:
# Least expensive mean - By price - Wallonia
low5_price_Wallonia_mean = make_top_5(df_threshold_Wallonia, "municipalities", "Price", "mean", "bottom")
low5_price_Wallonia_mean

,Zip_code,municipality,Price_mean
1,6020,Dampremy,129800
2,7340,Colfontaine,130429
3,7390,Quaregnon,138768
4,6200,Châtelineau,149582
5,6870,Saint-Hubert,152250


In [438]:
# Least expensive median - By price - Wallonia
low5_price_Wallonia_median = make_top_5(df_threshold_Wallonia, "municipalities", "Price", "median", "bottom")
low5_price_Wallonia_median

,Zip_code,municipality,Price_median
1,6020,Dampremy,109950
2,7340,Colfontaine,110000
3,6870,Saint-Hubert,122500
4,7390,Quaregnon,124500
5,7370,Dour,141250


In [439]:
# Least expensive mean - By € / SQM - Wallonia
low5_sqm_Wallonia_mean = make_top_5(df_threshold_Wallonia, "municipalities", "Price_per_square_meter", "mean", "bottom")
low5_sqm_Wallonia_mean

,Zip_code,municipality,Price_per_square_meter
1,6020,Dampremy,733
2,6870,Saint-Hubert,802
3,6590,Momignies,868
4,7370,Dour,872
5,6690,Vielsalm,877


In [440]:
# Clean zips dataframe
zips_full = zips_full.drop(axis=1, columns=["country_code", "admin_name1","admin_code1",'admin_name2 admin_code2','admin_name3', 'admin_code3', 'admin_code3.1', "accuracy"])
print(zips_full.columns)

Index(['Zip_code', 'place_name', 'lat', 'long'], dtype='object')


In [441]:
# Merge zips with house_df
one_df = df_threshold.merge(zips_full.drop_duplicates("Zip_code"), left_on="municipalities", right_on= "Zip_code", how="left")

In [442]:
# Plot mean price on map of Belgium
one_df_grouped = one_df.groupby("municipalities").agg({"Price": "mean"})
one_df_grouped.reset_index(level=0, inplace=True)
one_df_grouped_mean = one_df_grouped.merge(zips_full, how="left", right_on="Zip_code", left_on="municipalities")
one_df_grouped_mean = one_df_grouped_mean.sort_values("Price")
one_df_grouped_mean_10 = one_df_grouped_mean.iloc[[0,1,2,3,4,5,6,7,8,9,-9,-8,-7,-6,-5,-4,-3,-2,-1]]


fig = px.scatter_mapbox(one_df_grouped_mean, lat="lat", lon="long", hover_name="place_name",
                        zoom=6, color="Price",center={"lat":50.6, "lon":5.1},
                        color_continuous_scale="magenta",range_color=[0,1400000], size= "Price", size_max= 7 , opacity=0.9,
                        width=800, height=650)

fig.update_geos(fitbounds="locations")
fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":10,"l":0,"b":0}, geo=dict(projection_scale=4))
fig.update_layout(showlegend = True)
fig.show()

In [443]:
# # Plot median price on map of Belgium
one_df_grouped_median = one_df.groupby("municipalities").agg({"Price": "median"})
one_df_grouped_median.reset_index(level=0, inplace=True)
one_df_grouped_median = one_df_grouped_median.merge(zips_full, how="left", right_on="Zip_code", left_on="municipalities")
one_df_grouped_median = one_df_grouped_median.sort_values("Price")
one_df_grouped_median_10 = one_df_grouped_median.iloc[[0,1,2,3,4,5,6,7,8,9,-9,-8,-7,-6,-5,-4,-3,-2,-1]]

fig = px.scatter_mapbox(one_df_grouped_median, lat="lat", lon="long", hover_name="place_name", hover_data=["Zip_code"],
                        zoom=6, width=900, height=700, color="Price",center={"lat":50.6, "lon":5.1},
                        color_continuous_scale="magenta",range_color=[0,1200000], size= "Price", size_max= 7, opacity=0.9)

fig.update_geos(fitbounds="locations")
fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":10,"t":0,"l":0,"b":0}, geo=dict(projection_scale=4))
fig.update_layout(showlegend = True)
fig.show()

In [444]:
# Plot mean price per square_meter on map of Belgium
one_df_grouped_sqm = one_df.groupby("municipalities").agg({"Price_per_square_meter": "mean"})
one_df_grouped_sqm.reset_index(level=0, inplace=True)
one_df_grouped_sqm = one_df_grouped_sqm.merge(zips_full, how="left", right_on="Zip_code", left_on="municipalities")
one_df_grouped_sqm = one_df_grouped_sqm.sort_values("Price_per_square_meter")
one_df_grouped_sqm_10 = one_df_grouped_sqm.iloc[[0,1,2,3,4,5,6,7,8,9,-9,-8,-7,-6,-5,-4,-3,-2,-1]]

fig = px.scatter_mapbox(one_df_grouped_sqm, lat="lat", lon="long", hover_name="place_name", hover_data=["Zip_code"],
                        zoom=6, width=900, height=650, color="Price_per_square_meter",center={"lat":50.6, "lon":5.1},
                        color_continuous_scale="magenta",range_color=[0,4000], size= "Price_per_square_meter", size_max= 7, opacity=0.9)

fig.update_geos(fitbounds="locations")
fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":10,"t":0,"l":0,"b":0}, geo=dict(projection_scale=4))
fig.update_layout(showlegend = True)
fig.show()